In [523]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

# ✅ 데이터 불러오기
df = pd.read_csv("data/student-por.csv")

# ✅ `G1`, `G2` 변환 (평균 성적, 비율 계산)
df["G_avg"] = (df["G1"] + df["G2"]) / 2
df["G_ratio"] = df["G1"] / (df["G2"] + 1)  # 0으로 나누는 것 방지

# ✅ **6개월 기준 누적 공부 시간과 자유 시간 계산 (6개월 = 26주)**
df["studytime_per_6months"] = df["studytime"] * 26  # 하루 공부 시간을 6개월 단위로 변환
df["freetime_per_6months"] = df["freetime"] * 26  # 자유 시간도 6개월 단위로 변환
df["total_hours"] = df["studytime_per_6months"] + df["freetime_per_6months"]

# ✅ **공부 시간이 많을수록 성적 상승 효과 증가 (꾸준히 증가)**
df["studytime_boost"] = np.log(1 + df["studytime_per_6months"]) * 2  

# ✅ **자유 시간이 많을수록 성적 하락 패널티 적용 (완만한 감소)**
df["freetime_penalty"] = np.sqrt(df["freetime_per_6months"]) * 1.2  

# ✅ **학습 시간 비율 조정 (꾸준히 증가)**
df["studytime_ratio"] = (df["studytime_per_6months"] / (df["total_hours"] + 1)) ** 1.2  

# ✅ 학습 데이터 구성
X = df[['studytime_per_6months', 'freetime_per_6months', 'G_avg', 'G_ratio', 
        'total_hours', 'studytime_boost', 'freetime_penalty', 'studytime_ratio']]
y = df[['G3']]  # 성적(G3)

# ✅ MinMaxScaler 적용 (X는 정규화, y는 그대로 유지)
scaler_X = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# ✅ Linear Regression 학습
regressor_lr = LinearRegression()
regressor_lr.fit(X_train_scaled, y_train)

# ✅ XGBoost 학습 (🔥 G3 원래 점수 그대로 학습)
regressor_xgb = XGBRegressor(
    n_estimators=500,  
    learning_rate=0.03,  
    max_depth=5,  
    subsample=0.9,  
    colsample_bytree=0.9,  
    random_state=42
)
regressor_xgb.fit(X_train_scaled, y_train.values.ravel())

# ✅ 모델 평가
y_pred_train_lr = regressor_lr.predict(X_train_scaled)
y_pred_test_lr = regressor_lr.predict(X_test_scaled)
train_score_lr = r2_score(y_train, y_pred_train_lr) * 100
test_score_lr = r2_score(y_test, y_pred_test_lr) * 100

y_pred_train_xgb = regressor_xgb.predict(X_train_scaled)
y_pred_test_xgb = regressor_xgb.predict(X_test_scaled)
train_score_xgb = r2_score(y_train, y_pred_train_xgb) * 100
test_score_xgb = r2_score(y_test, y_pred_test_xgb) * 100

print(f"✅ [Linear Regression] 훈련 데이터 R²: {train_score_lr:.1f}%")
print(f"✅ [Linear Regression] 테스트 데이터 R²: {test_score_lr:.1f}%")
print(f"✅ [XGBoost] 훈련 데이터 R²: {train_score_xgb:.1f}%")
print(f"✅ [XGBoost] 테스트 데이터 R²: {test_score_xgb:.1f}%")

# ✅ 모델 비교 후 더 나은 모델 저장
if test_score_xgb > test_score_lr:
    joblib.dump(regressor_xgb, "regressor.pkl")  # ✅ XGBoost가 더 정확하면 저장
    best_model_name = "XGBoost"
else:
    joblib.dump(regressor_lr, "regressor.pkl")  # ✅ Linear Regression이 더 정확하면 저장
    best_model_name = "Linear Regression"

# ✅ MinMaxScaler 저장 (입력 데이터 정규화에 필요)
joblib.dump(scaler_X, "scaler_X.pkl")

print(f"🎯 ✅ {best_model_name} 모델이 저장되었습니다! (best_model.pkl)")


✅ [Linear Regression] 훈련 데이터 R²: 83.8%
✅ [Linear Regression] 테스트 데이터 R²: 83.0%
✅ [XGBoost] 훈련 데이터 R²: 96.0%
✅ [XGBoost] 테스트 데이터 R²: 77.7%
🎯 ✅ Linear Regression 모델이 저장되었습니다! (best_model.pkl)
